In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname))
        break
    

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/follicular-adenoma-sample/1028034_files/20.0
/kaggle/input/follicular-adenoma-sample/1028123_files/20.0
/kaggle/input/follicular-adenoma-sample/1028041_files/20.0
/kaggle/input/follicular-adenoma-sample/1028031_files/20.0
/kaggle/input/follicular-adenoma-sample/1028032_files/20.0
/kaggle/input/follicular-adenoma-sample/1028040_files/20.0
/kaggle/input/follicular-adenoma-sample/1028035_files/20.0
/kaggle/input/follicular-adenoma-sample/1028120_files/20.0
/kaggle/input/follicular-adenoma-sample/1028029_files/20.0
/kaggle/input/follicular-adenoma-sample/1028124_files/20.0
/kaggle/input/follicular-adenoma-sample/1028115_files/20.0
/kaggle/input/follicular-adenoma-sample/1028114_files/20.0
/kaggle/input/follicular-adenoma-sample/1028043_files/20.0
/kaggle/input/follicular-adenoma-sample/1028125_files/20.0
/kaggle/input/follicular-adenoma-sample/1028033_files/20.0
/kaggle/input/follicular-adenoma-sample/1028130_files/20.0
/kaggle/input/follicular-adenoma-sample/1028132_files/20

In [3]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class TumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        for dir_name in os.listdir(root_dir):
            if dir_name.endswith('_files'):
                dir_path = os.path.join(root_dir, dir_name, '20.0')
                label = 0 if int(dir_name.split('_')[0]) < 1028100 else 1  # 0: malignant, 1: benign
                for img_name in os.listdir(dir_path):
                    if img_name.endswith('.jpg'):
                        img_path = os.path.join(dir_path, img_name)
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [8]:
dataset = TumorDataset(root_dir='/kaggle/input/follicular-adenoma-sample')

In [9]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img.numpy()  # convert from tensor to numpy for plotting
    plt.imshow(np.transpose(img, (1, 2, 0)))  # re-order the dimensions from CxHxW to HxWxC
    plt.axis('off')  # remove axes for better visualization

def show_sample_images(dataset, num_images=6):
    fig = plt.figure(figsize=(15, 6))
    for i in range(num_images):
        idx = np.random.randint(0, len(dataset))  # Pick a random image
        img, label = dataset[idx]
        label_str = 'Malignant' if label == 0 else 'Benign'
        ax = fig.add_subplot(2, num_images//2, i + 1, xticks=[], yticks=[])
        imshow(img)
        ax.set_title(label_str)

show_sample_images(dataset)

ValueError: high <= 0

<Figure size 1500x600 with 0 Axes>

In [18]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class TumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        for dir_name in os.listdir(root_dir):
            if dir_name.endswith('_files'):  # Check if the directory name ends with '_files'
                dir_path = os.path.join(root_dir, dir_name)
                label = 0 if int(dir_name.split('_')[0]) < 1028100 else 1  # 0: malignant, 1: benign
                for img_name in os.listdir(dir_path):
                    if img_name.endswith('.jpg'):  # Check for .jpg files
                        img_path = os.path.join(dir_path, img_name)
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [19]:
import matplotlib.pyplot as plt
import numpy as np

# Transformation for displaying images
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize for display purposes
    transforms.ToTensor()
])

# Initialize dataset
dataset = TumorDataset(root_dir='/kaggle/input/follicular-adenoma-sample', transform=transform)


In [15]:
len(dataset)

0

In [11]:

def imshow(img):
    img = img.numpy()
    plt.imshow(np.transpose(img, (1, 2, 0)))
    plt.axis('off')

def show_sample_images(dataset, num_images=6):
    fig = plt.figure(figsize=(15, 6))
    for i in range(num_images):
        idx = np.random.randint(0, len(dataset))
        img, label = dataset[idx]
        label_str = 'Malignant' if label == 0 else 'Benign'
        ax = fig.add_subplot(1, num_images, i + 1, xticks=[], yticks=[])
        imshow(img)
        ax.set_title(label_str)

show_sample_images(dataset)


ValueError: high <= 0

<Figure size 1500x600 with 0 Axes>

# Step 2: Model Design

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: malignant, benign

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


# Step 3: Training

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to simplify the problem
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

dataset = TumorDataset(root_dir='follicular-adenoma-sample', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop (simplified)
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}")

print('Finished Training')


In [6]:
from torch.utils.data import random_split
dataset = TumorDataset('/kaggle/input/follicular-adenoma-sample')
# Assuming 'dataset' is your instance of TumorDataset
train_size = int(0.8 * len(dataset))  # 80% of the dataset for training
test_size = len(dataset) - train_size  # 20% of the dataset for testing
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


ValueError: num_samples should be a positive integer value, but got num_samples=0